In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import subprocess
import imageio
import multiprocessing
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import insightface
from insightface.app import FaceAnalysis
from tqdm.notebook import tqdm

In [ ]:
print('insightface', insightface.__version__)
print('numpy', np.__version__)

In [ ]:
# Initialise the face analysis app and face swapper model
def initialise_app_and_swapper():
    app = FaceAnalysis(name='buffalo_l')
    app.prepare(ctx_id=0, det_size=(640, 640))

    model_path = '/Users/mizz/Library/CloudStorage/OneDrive-UniversityofBristol/Portfolio/Personal/Creative_Hub/Tools/inswapper_128.onnx'
    swapper = insightface.model_zoo.get_model(model_path, download=False, download_zip=False)

    return app, swapper

# Process a single frame
def process_frame(frame, timestamp, face1, app, swapper):
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    face2_result = app.get(frame_bgr)
    if face2_result is None or not face2_result:
        return None, None, timestamp
    face2 = face2_result[0]
    frame_swapped = swapper.get(frame_bgr, face2, face1, paste_back=True)
    frame_swapped_rgb = cv2.cvtColor(frame_swapped, cv2.COLOR_BGR2RGB)
    return frame_swapped_rgb, True, timestamp

# Main function to swap faces in a video
def swap_n_show(img1_fn, video_fn, app, swapper):
    img1 = cv2.imread(img1_fn)
    face1 = app.get(img1)[0]
    video = imageio.get_reader(video_fn)
    fps = video.get_meta_data()['fps']

    cpu_count = multiprocessing.cpu_count()
    with ThreadPoolExecutor(max_workers=cpu_count) as executor:
        futures = []
        for i, frame in enumerate(video):
            timestamp = i / fps  # Calculate the timestamp of each frame
            futures.append(executor.submit(process_frame, frame, timestamp, face1, app, swapper))

        results = []
        for future in tqdm(futures, total=len(futures), desc='Processing frames'):
            results.append(future.result())

    swapped_video, status_list, timestamps = zip(*results)
    # Filter out frames and collect timestamps of kept frames
    kept_frames_info = [(frame, timestamp) for frame, status, timestamp in zip(swapped_video, status_list, timestamps) if status]

    if kept_frames_info:
        swapped_video, kept_timestamps = zip(*kept_frames_info)
        handle_video_processing(video_fn, swapped_video, fps, kept_timestamps)

def adjust_audio_track(video_fn, kept_timestamps, fps, audio_output_fn):
    # Temporary directory to store individual audio segments
    temp_dir = Path("temp_audio_segments")
    temp_dir.mkdir(exist_ok=True)

    ffmpeg_path = '/Users/mizz/Library/CloudStorage/OneDrive-UniversityofBristol/Portfolio/Personal/Creative_Hub/Tools/ffmpeg'

    # List to keep track of the segment filenames
    segment_filenames = []

    # Extract individual audio segments based on kept timestamps
    for i, timestamp in enumerate(kept_timestamps):
        start_time = timestamp
        try:
            end_time = kept_timestamps[i + 1]
        except IndexError:
            end_time = None  # Handle the last segment
        
        segment_fn = temp_dir / f"segment_{i}.mp3"
        segment_filenames.append(segment_fn)

        ffmpeg_cmd = [ffmpeg_path, '-i', video_fn, '-ss', str(start_time)]
        if end_time:
            duration = end_time - start_time
            ffmpeg_cmd.extend(['-t', str(duration)])
        ffmpeg_cmd.extend(['-vn', '-acodec', 'copy', str(segment_fn)])
        subprocess.run(ffmpeg_cmd)

    # Concatenate all segments
    concat_list_fn = temp_dir / "concat_list.txt"
    with open(concat_list_fn, 'w') as f:
        for segment_fn in segment_filenames:
            f.write(f"file '{segment_fn}'\n")

    # Use ffmpeg to concatenate the audio segments
    subprocess.run([ffmpeg_path, '-f', 'concat', '-safe', '0', '-i', concat_list_fn, '-c', 'copy', audio_output_fn])

    # Clean up: remove temporary files and directory
    for segment_fn in segment_filenames:
        segment_fn.unlink()
    concat_list_fn.unlink()
    temp_dir.rmdir()

def handle_video_processing(video_fn, swapped_video, fps, kept_timestamps):
    ffmpeg_path = '/Users/mizz/Library/CloudStorage/OneDrive-UniversityofBristol/Portfolio/Personal/Creative_Hub/Tools/ffmpeg'
    input_path = Path(video_fn)
    extension = input_path.suffix.lower()
    if extension in ['.mp4', '.mov']:
        video_codec = 'libx264'
        audio_codec = 'aac'
        audio_extension = '.m4a'  # Use m4a for aac audio
        output_extension = '.mp4'
    elif extension == '.webm':
        video_codec = 'libvpx-vp9'
        audio_codec = 'libvorbis'
        audio_extension = '.ogg'
        output_extension = '.mp4'  # Convert to mp4
    else:
        raise ValueError("Unsupported video format")
    
    # Generate the output video file name
    output_video_fn = str(input_path.stem) + '_output' + output_extension
    output_path = Path.cwd() / output_video_fn
    
    # Write the frames to the output video file
    with imageio.get_writer(output_path, fps=fps, codec=video_codec, quality=9, ffmpeg_params=['-vf', 'scale=trunc(iw/2)*2:trunc(ih/2)*2']) as writer:
        for frame in swapped_video:
            if frame is not None:
                writer.append_data(frame)

    # Extract audio from the original video
    audio_fn = str(input_path.stem) + '_audio' + audio_extension
    subprocess.run([ffmpeg_path, '-i', video_fn, '-vn', '-acodec', audio_codec, audio_fn, '-y'])

    # Create a new audio track that matches the kept video frames
    adjust_audio_track(video_fn, kept_timestamps, fps, audio_fn)

    # Combine the new video with the original audio
    final_output_fn = str(input_path.stem) + '_final' + output_extension
    final_output_path = Path.cwd() / final_output_fn
    subprocess.run([ffmpeg_path, '-i', output_path, '-i', audio_fn, '-c:v', 'copy', '-c:a', 'copy', '-shortest', final_output_path, '-y'])

    # Clean up the intermediate files
    os.remove(output_path)
    os.remove(audio_fn)

    plt.figure(figsize=(5, 5))
    plt.imshow(swapped_video[0])
    plt.axis('off')
    plt.show()

    print(f"Final output saved to {final_output_path}")

def main():
    app, swapper = initialise_app_and_swapper()
    os.environ['IMAGEIO_FFMPEG_EXE'] = '/Users/mizz/Library/CloudStorage/OneDrive-UniversityofBristol/Portfolio/Personal/Creative_Hub/Tools/ffmpeg'
    
    image_path = 'yeji.jpg'
    video_path = 'Source/Videos/'
    
    # List of video file names to process
    # video_files = [f'video_files_{i}.mp4' for i in range(1, 10)]

    # Process each video file
    for video_file in video_files:
        print(f"Processing {video_file}...")
        full_video_path = os.path.join(video_path, video_file)
        try:
            swap_n_show(image_path, full_video_path, app, swapper)
            print(f"Finished processing {video_file}")
        except Exception as e:
            print(f"Failed to process {video_file}: {e}")

if __name__ == "__main__":
    main()